

```
██╗   ██╗███╗   ██╗██╗ ██████╗ ██╗   ██╗███████╗     ██████╗ ██████╗ ██╗      █████╗ ██████╗
██║   ██║████╗  ██║██║██╔═══██╗██║   ██║██╔════╝    ██╔════╝██╔═══██╗██║     ██╔══██╗██╔══██╗
██║   ██║██╔██╗ ██║██║██║   ██║██║   ██║█████╗      ██║     ██║   ██║██║     ███████║██████╔╝
██║   ██║██║╚██╗██║██║██║▄▄ ██║██║   ██║██╔══╝      ██║     ██║   ██║██║     ██╔══██║██╔══██╗
╚██████╔╝██║ ╚████║██║╚██████╔╝╚██████╔╝███████╗    ╚██████╗╚██████╔╝███████╗██║  ██║██████╔╝
 ╚═════╝ ╚═╝  ╚═══╝╚═╝ ╚══▀▀═╝  ╚═════╝ ╚══════╝     ╚═════╝ ╚═════╝ ╚══════╝╚═╝  ╚═╝╚═════╝
                                                                                             
```
🚀 **Minecraft Server Setup on Google Colab** 🚀

🔥 Made by @ItsUniquePlayz (YouTube) 🔥

💻 GitHub: https://github.com/ItsUniquePlayz/UniqueColab

🌍 Create your own free server & go online instantly!


## 📌 **Part 1: Setting Up the Environment**

In [ ]:
import os
import json
import requests
import subprocess
from google.colab import drive

# 📂 Google Drive Mount
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive', force_remount=True)

# 📁 Create Server Directory
server_dir = "/content/drive/MyDrive/MinecraftServer"
os.makedirs(server_dir, exist_ok=True)
os.chdir(server_dir)

# 📝 Create Configuration File
config = {"server_type": "paper", "server_version": "1.21.1"}
with open("server_config.json", "w") as config_file:
    json.dump(config, config_file)

# 🛠 Install Java 21
print("🔧 Installing Java 21...")
subprocess.run(["sudo", "apt", "update", "-qq"])
subprocess.run(["sudo", "apt", "install", "-y", "openjdk-21-jre-headless", "-qq"])
print("✅ Java 21 installed successfully!")

# 🎮 Download the Latest PaperMC
print("📥 Fetching the latest PaperMC version...")
paper_api_url = "https://api.papermc.io/v2/projects/paper/versions/1.21.1"
response = requests.get(paper_api_url)

if response.status_code == 200:
    latest_build = response.json()["builds"][-1]
    paper_download_url = f"https://api.papermc.io/v2/projects/paper/versions/1.21.1/builds/{latest_build}/downloads/paper-1.21.1-{latest_build}.jar"
    jar_name = "server.jar"

    jar_response = requests.get(paper_download_url)
    if jar_response.status_code == 200:
        with open(jar_name, "wb") as file:
            file.write(jar_response.content)
        print("✅ PaperMC Download Complete!")
    else:
        print(f"❌ PaperMC download failed! Error {jar_response.status_code}")
else:
    print("❌ Failed to fetch PaperMC build details.")

# 📜 Accept Minecraft EULA
with open("eula.txt", "w") as eula_file:
    eula_file.write("eula=true\n")
print("✅ EULA Accepted.")

# 📂 Create Plugins Folder & Install Playit.gg Plugin
plugins_dir = os.path.join(server_dir, "plugins")
os.makedirs(plugins_dir, exist_ok=True)
playit_plugin_url = "https://github.com/playit-cloud/playit-minecraft-plugin/releases/latest/download/playit-minecraft-plugin.jar"
plugin_path = os.path.join(plugins_dir, "playit-minecraft-plugin.jar")

print("📥 Downloading Playit.gg plugin...")
plugin_response = requests.get(playit_plugin_url)
if plugin_response.status_code == 200:
    with open(plugin_path, "wb") as file:
        file.write(plugin_response.content)
    print("✅ Playit.gg plugin downloaded successfully!")
else:
    print("❌ Failed to download Playit.gg plugin.")

print("🚀 Setup Complete! Run Part 2 to start the server.")


##**📌 Part 2: Installing Java & Running the Server**

In [ ]:
import os
import json
import subprocess
import threading
import sys

# 📜 Load Configuration File
config_path = "server_config.json"
if not os.path.isfile(config_path):
    print("⚠️ Config file not found! Please run Part 1 first.")
    exit()

with open(config_path, "r") as config_file:
    config = json.load(config_file)

# 📂 Change to Server Directory
server_dir = "/content/drive/MyDrive/MinecraftServer"
os.chdir(server_dir)

# 🔍 Verify Java Installation
try:
    java_check = subprocess.run(["java", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    java_output = java_check.stdout + java_check.stderr

    if "21." not in java_output:
        raise ValueError("❌ Java 21 not found! Please install it in Part 1.")
    print("✅ Java 21 is installed correctly!")
except FileNotFoundError:
    print("❌ Java is not installed! Please install it in Part 1.")
    exit()

# 🎮 Check if server.jar exists
jar_name = "server.jar"
if not os.path.isfile(jar_name):
    print("❌ server.jar not found! Please download it in Part 1.")
    exit()

# 📂 Check if Playit.gg Plugin is Installed
plugins_dir = os.path.join(server_dir, "plugins")
plugin_path = os.path.join(plugins_dir, "playit-minecraft-plugin.jar")
if not os.path.isfile(plugin_path):
    print("❌ Playit.gg plugin not found! Please download it manually to the 'plugins' folder.")
else:
    print("✅ Playit.gg plugin is installed!")

# ⚙️ Set Server Flags & Start Server
memory_allocation = "-Xms4G -Xmx8G"
server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch"

print(f"🚀 Starting Minecraft {config['server_version']} server...")

server_process = subprocess.Popen(
    ["java"] + memory_allocation.split() + server_flags.split() + ["-jar", jar_name, "nogui"],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

# 🔁 Function to Show Live Console Output
def read_output(process):
    while True:
        output = process.stdout.readline()
        if output:
            print(output.strip())  # Show live output
        if process.poll() is not None:
            break

# 🔁 Start Console Thread
console_thread = threading.Thread(target=read_output, args=(server_process,))
console_thread.daemon = True
console_thread.start()

# 🖥️ Allow User to Input Commands in Console
print("✅ Server is running! Type your commands below:")

while True:
    try:
        command = input("⏩ Enter Command: ")
        if command.lower() == "exit":
            print("❌ Stopping server...")
            server_process.stdin.write("stop\n")
            server_process.stdin.flush()
            server_process.wait()
            break
        else:
            server_process.stdin.write(command + "\n")
            server_process.stdin.flush()
    except KeyboardInterrupt:
        print("\n❌ Stopping server...")
        server_process.stdin.write("stop\n")
        server_process.stdin.flush()
        server_process.wait()
        break


##🌍 Connect With Me! 🚀🎮
🔥 YouTube: https://www.youtube.com/@ItsUniquePlayz – Minecraft Server Tutorials & More!

💻 GitHub: https://github.com/ItsUniquePlayz – Get the latest code & updates!

📸 Instagram: https://www.instagram.com/itsuniqueplayz/ – Behind the scenes & fun content!

💬 Discord: http://discordapp.com/users/1310302434241024102 – Join the community & ask questions!

🚀 Subscribe, Follow & Stay Connected for More Awesome Content! 🔥